# Mini Competition: Richter's Predictor

Import modules and data

In [36]:
# Modules
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [37]:
# Import scripts
import helper_functions # various helper functions
import log_regression # Simple regression model
import lgb_optimized # Random forest

In [38]:
# Load data
X, y, X_test = helper_functions.imports()

### Data Analysis

Start with a basic analysis of the data to get a general feeling of what we are dealing with.

In [39]:
print(f"Proportions of the DataFrame X, containing the features for testing: {X.shape}")
print(f"Proportions of the DataFrage y, containing the target value for testing: {y.shape}")
print(f"Proportions of the DataFrame X, containing the features for the prediction: {X_test.shape}")

Proportions of the DataFrame X, containing the features for testing: (260601, 39)
Proportions of the DataFrage y, containing the target value for testing: (260601, 2)
Proportions of the DataFrame X, containing the features for the prediction: (86868, 39)


In [40]:
X.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,0,0,0,0,0,0
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,0,0,0,0,0,0
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,0,0,0,0,0,0


In [41]:
y.head()

,building_id,damage_grade
0,802906,3
1,28830,2
2,94947,3
3,590882,2
4,201944,3


In [42]:
X_test.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,300051,17,596,11307,3,20,7,6,t,r,...,0,0,0,0,0,0,0,0,0,0
1,99355,6,141,11987,2,25,13,5,t,r,...,1,0,0,0,0,0,0,0,0,0
2,890251,22,19,10044,2,5,4,5,t,r,...,0,0,0,0,0,0,0,0,0,0
3,745817,26,39,633,1,0,19,3,t,r,...,0,0,1,0,0,0,0,0,0,0
4,421793,17,289,7970,3,15,8,7,t,r,...,0,0,0,0,0,0,0,0,0,0


Check whether X and X_test have the same columns

In [43]:
X.columns

Index(['building_id', 'geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_i

In [44]:
helper_functions.test_column_equality(X, X_test)

Both DataFrames have the same columns.


In [45]:
X.dtypes

building_id                                int64
geo_level_1_id                             int64
geo_level_2_id                             int64
geo_level_3_id                             int64
count_floors_pre_eq                        int64
age                                        int64
area_percentage                            int64
height_percentage                          int64
land_surface_condition                    object
foundation_type                           object
roof_type                                 object
ground_floor_type                         object
other_floor_type                          object
position                                  object
plan_configuration                        object
has_superstructure_adobe_mud               int64
has_superstructure_mud_mortar_stone        int64
has_superstructure_stone_flag              int64
has_superstructure_cement_mortar_stone     int64
has_superstructure_mud_mortar_brick        int64
has_superstructure_c

In [46]:
X.nunique()

building_id                               260601
geo_level_1_id                                31
geo_level_2_id                              1414
geo_level_3_id                             11595
count_floors_pre_eq                            9
age                                           42
area_percentage                               84
height_percentage                             27
land_surface_condition                         3
foundation_type                                5
roof_type                                      3
ground_floor_type                              5
other_floor_type                               4
position                                       4
plan_configuration                            10
has_superstructure_adobe_mud                   2
has_superstructure_mud_mortar_stone            2
has_superstructure_stone_flag                  2
has_superstructure_cement_mortar_stone         2
has_superstructure_mud_mortar_brick            2
has_superstructure_c

From the description, we assume Geo Level 3 is the most precise whereas Geo Level 1 the least precise.
Follow that logic, we expect more unique data points in Level 3 than Level 1.

In [47]:
print(f"Unique data points in geo_level_1_id: {X.loc[:, 'geo_level_1_id'].nunique()}")
print(f"Unique data points in geo_level_2_id: {X.loc[:, 'geo_level_2_id'].nunique()}")
print(f"Unique data points in geo_level_3_id: {X.loc[:, 'geo_level_3_id'].nunique()}")

Unique data points in geo_level_1_id: 31
Unique data points in geo_level_2_id: 1414
Unique data points in geo_level_3_id: 11595


## Data Cleaning

In [48]:
# list of columns that shall be dropped at the end
drop_cols = []

# helper functions
def ids():
    ids = [id for id in globals().keys() if not id.startswith("_")]
    print(ids[5:])
ids()

print(X.shape, 'damage_grade' in X.columns)
data = pd.merge(left=y, right=X, on='building_id', how='inner')
print(data.shape, 'damage_grade' in data.columns) #ok, one column was added

cols = data.columns
for col in cols:
    globals()[col] = col



['open', 'pd', 'train_test_split', 'f1_score', 'helper_functions', 'log_regression', 'lgb_optimized', 'X', 'y', 'X_test', 'drop_cols', 'ids', 'data', 'cols', 'col', 'building_id', 'damage_grade', 'geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id', 'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage', 'land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type', 'other_floor_type', 'position', 'plan_configuration', 'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag', 'has_superstructure_cement_mortar_stone', 'has_superstructure_mud_mortar_brick', 'has_superstructure_cement_mortar_brick', 'has_superstructure_timber', 'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered', 'has_superstructure_rc_engineered', 'has_superstructure_other', 'legal_ownership_status', 'count_families', 'has_secondary_use', 'has_secondary_use_agriculture', 'has_secondary_use_hotel', 'has_secondary_use_rental', 'has_

### `geo_level2` mean encodings

In [49]:
geo_levels = []
for col in filter(lambda col: col.startswith("geo"), X):
    print(str(col))
    geo_levels.append(col)

geo_level_1_id
geo_level_2_id
geo_level_3_id


In [50]:
mean_encodings = data.groupby('geo_level_2_id')['damage_grade'].mean()
pd.DataFrame(mean_encodings).head()

,damage_grade
geo_level_2_id,
0,2.763158
1,2.348039
3,2.103896
4,2.520635
5,2.160000


In [51]:
for df in X, X_test:
    df["geo_level_2_enc"] = df['geo_level_2_id'].map(mean_encodings)
    print(df[["geo_level_2_id", "geo_level_2_enc"]].sort_values("geo_level_2_id")[:100], end="\n")

print(geo_levels)
drop_cols.extend(geo_levels)

        geo_level_2_id  geo_level_2_enc
42505                0         2.763158
137871               0         2.763158
222351               0         2.763158
132375               0         2.763158
11043                0         2.763158
...                ...              ...
212321               1         2.348039
108873               1         2.348039
173444               1         2.348039
100864               1         2.348039
191316               1         2.348039

[100 rows x 2 columns]
       geo_level_2_id  geo_level_2_enc
30583               0         2.763158
40508               0         2.763158
20848               0         2.763158
54201               0         2.763158
66573               0         2.763158
...               ...              ...
21571               4         2.520635
8492                4         2.520635
10806               4         2.520635
66004               4         2.520635
29765               4         2.520635

[100 rows x 2 columns]
['ge

### drop categorical variables for now, I will encode them next

In [52]:
object_dtype_cols = [
    "land_surface_condition", "foundation_type", "roof_type", "ground_floor_type", 
    "other_floor_type", "position", "plan_configuration", "legal_ownership_status"
]

drop_cols.extend(object_dtype_cols)

In [53]:
# Outliers 

### Drop columns and split data

In [54]:
for df in y, X, X_test:
    df.set_index("building_id", inplace=True)

y.head()
y.shape

(260601, 1)

In [55]:
print(drop_cols)
for df in X, X_test:
    df.drop(columns=drop_cols, inplace=True)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
X_test_pred = X_test
# X_train.columns

['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id', 'land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type', 'other_floor_type', 'position', 'plan_configuration', 'legal_ownership_status']


In [56]:
# # Split data in training and validation set
# y = y['damage_grade']
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)


# #FOR NOW (to test tree gen) limit dataset to a few parameters only
# mask = ['has_superstructure_adobe_mud', 'age','count_floors_pre_eq','area_percentage','height_percentage','has_secondary_use',
#         'has_superstructure_cement_mortar_brick', 'has_superstructure_timber', 'has_superstructure_bamboo'] 

# # apply mask
# X = X[mask]

# X_train = X_train[mask]
# X_valid = X_valid[mask]
# X_test_pred = X_test[mask]

Visualization

## Models

In [57]:
y.head()

,damage_grade
building_id,
802906,3
28830,2
94947,3
590882,2
201944,3


Simple estimation using regressionm

In [58]:
lr = log_regression.lr(X_train, y_train)
y_valid_pred_lr = lr.make_prediction(X_valid)

Generation of Random Forest and optimization

In [59]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [60]:
import lgb_optimized
# Initiate and optimize model
lgb = lgb_optimized.LGBM(X_train, X_valid, y_train, y_valid)

[I 2023-10-06 19:38:53,135] A new study created in memory with name: no-name-37bcb898-98b7-47ae-b488-3fbcf8854a67
[I 2023-10-06 19:39:00,117] Trial 0 finished with value: 0.15653959056810116 and parameters: {'learning_rate': 0.25817652798233975, 'subsample': 0.7081555063568187, 'num_leaves': 68, 'min_data_in_leaf': 19, 'max_depth': 3, 'lambda_l2': 0.2339587632286676}. Best is trial 0 with value: 0.15653959056810116.
[I 2023-10-06 19:39:12,370] Trial 1 finished with value: 0.15535005084323017 and parameters: {'learning_rate': 0.03616385377328596, 'subsample': 0.8226601837783403, 'num_leaves': 21, 'min_data_in_leaf': 3, 'max_depth': 7, 'lambda_l2': 0.3702614291317068}. Best is trial 0 with value: 0.15653959056810116.


Number of finished trials: 2
Best trial:
--------------------------------
Best F1 Score: 0.15653959056810116
--------------------------------
[LightGBM] [Warning] min_data_in_leaf is set=19, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=19
[LightGBM] [Warning] lambda_l2 is set=0.2339587632286676, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.2339587632286676
[LightGBM] [Warning] min_data_in_leaf is set=19, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=19
[LightGBM] [Warning] lambda_l2 is set=0.2339587632286676, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.2339587632286676
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.121799 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 450
[LightGBM] [Info] Number of data points in the train set: 260601, number of used features: 28
[LightGBM] [Info] Start training from score -2.339167
[Li

In [61]:
y_pred_lgb = lgb.make_prediction(X_valid)

[LightGBM] [Warning] min_data_in_leaf is set=19, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=19
[LightGBM] [Warning] lambda_l2 is set=0.2339587632286676, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.2339587632286676


#### Evaluation

f1 Score using Logistic Regression

In [62]:
f1_lr = f1_score(y_valid, y_valid_pred_lr, average='macro')
f1_lr

0.580890201171839

f1 Score for Random Forest

In [63]:
f1_lgb = f1_score(y_valid, y_pred_lgb, average='macro')
f1_lgb

0.6534159213293927

### Create prediction on testset

In [64]:
y_pred_lgb = lgb.make_prediction(X_test_pred)

[LightGBM] [Warning] min_data_in_leaf is set=19, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=19
[LightGBM] [Warning] lambda_l2 is set=0.2339587632286676, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.2339587632286676


Export

In [65]:
# Export prediction to create csv by DrivenData.org datastandards
X_test.reset_index(inplace=True)

helper_functions.write_output(X_test, y_pred_lgb)

In [66]:
X.columns

Index(['count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone',
       'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'count_families', 'has_secondary_use', 'has_secondary_use_agriculture',
       'has_secondary_use_hotel', 'has_secondary_use_rental',
       'has_secondary_use_institution', 'has_secondary_use_school',
       'has_secondary_use_industry', 'has_secondary_use_health_post',
       'has_secondary_use_gov_office', 'has_secondary_use_use_police',
       'has_secondary_use_other', 'geo_level_2_enc'],
      dtype='object')

In [24]:
import session_info
session_info.show()